In [2]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
String rhinoVersion = "1.7.15"

In [3]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /list-repos


Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Repositories count: 5
name: central url: https://repo.maven.apache.org/maven2/ release:true update:never snapshot:false update:never 
name: jcenter url: https://jcenter.bintray.com/ release:true update:never snapshot:false update:never 
name: jboss url: https://repository.jboss.org/nexus/content/repositories/releases/ release:true update:never snapshot:false update:never 
name: atlassian url: https://packages.atlassian.com/maven/public release:true update:never snapshot:false update:never 
name: local url: file:///Users/bsorrentino/.m2/repository/ release:true update:never snapshot:true update:always 


In [4]:
%dependency /add  org.mozilla:rhino:\{rhinoVersion}
%dependency /resolve

Adding dependency org.mozilla:rhino:1.7.15
Solving dependencies
Resolved artifacts count: 1
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/mozilla/rhino/1.7.15/rhino-1.7.15.jar


In [5]:
import org.mozilla.javascript.Context;
import org.mozilla.javascript.ContextAction;
import org.mozilla.javascript.ContextFactory;
import org.mozilla.javascript.ErrorReporter;
import org.mozilla.javascript.Function;
import org.mozilla.javascript.ImporterTopLevel;
import org.mozilla.javascript.NativeArray;
import org.mozilla.javascript.NativeConsole;
import org.mozilla.javascript.RhinoException;
import org.mozilla.javascript.Script;
import org.mozilla.javascript.ScriptRuntime;
import org.mozilla.javascript.Scriptable;
import org.mozilla.javascript.ScriptableObject;
import org.mozilla.javascript.Synchronizer;
import org.mozilla.javascript.Undefined;
import org.mozilla.javascript.Wrapper;
import org.mozilla.javascript.commonjs.module.Require;
import org.mozilla.javascript.commonjs.module.RequireBuilder;
import org.mozilla.javascript.commonjs.module.provider.SoftCachingModuleScriptProvider;
import org.mozilla.javascript.commonjs.module.provider.UrlModuleSourceProvider;
import org.mozilla.javascript.serialize.ScriptableInputStream;
import org.mozilla.javascript.serialize.ScriptableOutputStream;
import org.mozilla.javascript.tools.ToolErrorReporter;

public class CommonJsImporter extends ImporterTopLevel {

    public CommonJsImporter(Context ctx, boolean sealed) {
        super(ctx, sealed);
    }

    /**
     * fix problem for Typescript generation
     * Object.defineProperty(exports, "__esModule", { value: true });
     * @param object empty javascript object
     */
    public void defineES6Exports(Scriptable object) {
        ScriptableObject.putProperty(this, "exports", object);
    }

    public void installRequire(Context ctx, List<URI> uris, boolean sandboxed) {


        RequireBuilder rb = new RequireBuilder();
        rb.setSandboxed(sandboxed);
        rb.setModuleScriptProvider(
                new SoftCachingModuleScriptProvider(new UrlModuleSourceProvider(uris, null)));
        Require require = rb.createRequire(ctx, this);

        require.install(this);
    }


}


In [6]:
System.getProperty("user.dir");

/Users/bsorrentino/WORKSPACES/GITHUB.me/JAVA/java2typescript/samples/rhino

In [14]:
import org.mozilla.javascript.Context;
import org.mozilla.javascript.EvaluatorException;
import org.mozilla.javascript.Function;
import org.mozilla.javascript.JavaScriptException;
import org.mozilla.javascript.Scriptable;
import org.mozilla.javascript.ScriptableObject;
import org.mozilla.javascript.WrappedException;
import org.mozilla.javascript.ContextFactory;
import java.nio.file.Paths;

Object runScript( String js  ) throws Exception {   
    
    var factoryCtx = new ContextFactory();


    var cwd = Paths.get( System.getProperty("user.dir"), "src", "test", "resources" );
    var modules = List.of( cwd.toUri() );

    return factoryCtx.call( ctx -> {
        ctx.setLanguageVersion(Context.VERSION_ES6);
        
        CommonJsImporter topLevel = new CommonJsImporter(ctx, false);

        ctx.initStandardObjects(topLevel, false);
        topLevel.installRequire(ctx, modules, true);

        Scriptable newScope = ctx.newObject(topLevel);
        newScope.setPrototype(topLevel);

        return ctx.evaluateString(newScope, js, "<cmd>", 1, null);
    });

}


In [16]:
var js = """
        const test = require('test-cjs');
        test();
        """;
var result = runScript( js );

The CommonJS require function works!
